# Demo

## Setup

In [2]:
import pandas as pd
import multiprocessing as mp
import time

from src.paths import CACHE_DIR

In [3]:
data = pd.read_csv(CACHE_DIR / 'temperature_data.csv', parse_dates=['timestamp'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54750 entries, 0 to 54749
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   city         54750 non-null  object        
 1   timestamp    54750 non-null  datetime64[ns]
 2   temperature  54750 non-null  float64       
 3   season       54750 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.7+ MB


## Solution

### 1. **Анализ исторических данных**:
   - Вычислить **скользящее среднее** температуры с окном в 30 дней для сглаживания краткосрочных колебаний.
   - Рассчитать среднюю температуру и стандартное отклонение для каждого сезона в каждом городе.
   - Выявить аномалии, где температура выходит за пределы $ \text{среднее} \pm 2\sigma $.
   - Попробуйте распараллелить проведение этого анализа. Сравните скорость выполнения анализа с распараллеливанием и без него.



In [4]:
from src.utils import add_features_sequential, add_features_parallelized

In [5]:
num_processes = mp.cpu_count()

print(f'Number of available CPU cores: {num_processes}')

Number of available CPU cores: 12


In [6]:
# Run analysis without parallelization and measure time
start_time = time.time()
df_sequential = add_features_sequential(data.copy())
sequential_time = time.time() - start_time

# Run analysis with parallelization and measure time
start_time = time.time()
df_parallel = add_features_parallelized(data.copy(), num_processes // 4)
parallel_time = time.time() - start_time

print(f"Sequential analysis took: {sequential_time:.4f}s")
print(f"Parallel analysis took: {parallel_time:.4f}s")
print(f"Speedup: {sequential_time / parallel_time:.2f}x")

Sequential analysis took: 0.0395s
Parallel analysis took: 0.7513s
Speedup: 0.05x


In [7]:
print("Results with sequential analysis:")
display(df_sequential.head())
print("Results with parallel analysis:")
display(df_parallel.head())

Results with sequential analysis:


,city,timestamp,temperature,season,rolling_mean,seasonal_mean,seasonal_std,outlier
0,New York,2010-01-01,-0.687582,winter,-0.687582,0.188809,5.186381,False
1,New York,2010-01-02,-2.672985,winter,-1.680284,0.188809,5.186381,False
2,New York,2010-01-03,-0.808035,winter,-1.389534,0.188809,5.186381,False
3,New York,2010-01-04,-3.996199,winter,-2.041200,0.188809,5.186381,False
4,New York,2010-01-05,-5.341203,winter,-2.701201,0.188809,5.186381,False


Results with parallel analysis:


,city,timestamp,temperature,season,rolling_mean,seasonal_mean,seasonal_std,outlier
0,New York,2010-01-01,-0.687582,winter,-0.687582,0.188809,5.186381,False
1,New York,2010-01-02,-2.672985,winter,-1.680284,0.188809,5.186381,False
2,New York,2010-01-03,-0.808035,winter,-1.389534,0.188809,5.186381,False
3,New York,2010-01-04,-3.996199,winter,-2.041200,0.188809,5.186381,False
4,New York,2010-01-05,-5.341203,winter,-2.701201,0.188809,5.186381,False


### 2. **Мониторинг текущей температуры**:
   - Подключить OpenWeatherMap API для получения текущей температуры города. Для получения API Key (бесплатно) надо зарегистрироваться на сайте. Обратите внимание, что API Key может активироваться только через 2-3 часа, это нормально. Посему получите ключ заранее.
   - Получить текущую температуру для выбранного города через OpenWeatherMap API.
   - Определить, является ли текущая температура нормальной, исходя из исторических данных для текущего сезона.
   - Данные на самом деле не совсем реальные (сюрпрайз). Поэтому на момент эксперимента погода в Берлине, Каире и Дубае была в рамках нормы, а в Пекине и Москве аномальная. Протестируйте свое решение для разных городов.
   - Попробуйте для получения текущей температуры использовать синхронные и асинхронные методы. Что здесь лучше использовать?

In [14]:
from src.utils import get_current_temperature_sync, get_current_temperature_async, check_current_temperature


cities = ["Berlin", "Cairo", "Dubai", "Beijing", "Moscow"]


def main_sync():

    results = []
    for city in cities:
        start_time = time.time()
        current_temperature_sync = get_current_temperature_sync(city)
        sync_time = time.time() - start_time
        normal = check_current_temperature(df_sequential, city, current_temperature_sync)

        result = {
            'city': city,
            'current_temperature': current_temperature_sync,
            'normal': normal,
            'time': sync_time,
        }
        results.append(result)

    display(pd.DataFrame(results))


async def main_async():

    results = []
    for city in cities:
        start_time = time.time()
        current_temperature_async = await get_current_temperature_async(city)
        async_time = time.time() - start_time
        normal = check_current_temperature(df_sequential, city, current_temperature_async)

        result = {
            'city': city,
            'current_temperature': current_temperature_async,
            'normal': normal,
            'time': async_time,
        }
        results.append(result)

    display(pd.DataFrame(results))


print("Synchronous method:")
main_sync()

print("Asynchronous method:")
await main_async()

Synchronous method:


,city,current_temperature,normal,time
0,Berlin,5.64,True,0.144432
1,Cairo,21.42,True,0.144628
2,Dubai,21.96,True,0.148263
3,Beijing,-5.06,True,0.144182
4,Moscow,-1.78,True,0.145785


Asynchronous method:


,city,current_temperature,normal,time
0,Berlin,5.64,True,0.183516
1,Cairo,21.42,True,0.144257
2,Dubai,21.96,True,0.143298
3,Beijing,-5.06,True,0.155492
4,Moscow,-1.78,True,0.153689


### 3. **Создание приложения на Streamlit**:
   - Добавить интерфейс для загрузки файла с историческими данными.
   - Добавить интерфейс для выбора города (из выпадающего списка).
   - Добавить форму для ввода API-ключа OpenWeatherMap. Когда он не введен, данные для текущей погоды не показываются. Если ключ некорректный, выведите на экран ошибку (должно приходить `{"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}`).
   - Отобразить:
     - Описательную статистику по историческим данным для города, можно добавить визуализации.
     - Временной ряд температур с выделением аномалий (например, точками другого цвета).
     - Сезонные профили с указанием среднего и стандартного отклонения.
   - Вывести текущую температуру через API и указать, нормальна ли она для сезона.

In [ ]:
!streamlit run app.py